In [1]:
model = catalog.load('nb_model')

[04/05/23 19:58:55] INFO     Loading data from 'nb_model' (MlflowMlflowModelSaverDataSet)...    ]8;id=409216;file:///home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=643066;file:///home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [2]:
model

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(transformers=[('tfidf',
                                                                         TfidfVectorizer(),
                                                                         'text')])),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.1, 1, 5],
                         'clf__fit_prior': [True, False]},
             refit='f1_weighted',
             scoring=['f1_weighted', 'precision_weighted', 'recall_weighted'])

In [3]:
import shap

[04/05/23 19:59:08] WARNING  Importing display from IPython.core.display is deprecated since        ]8;id=542822;file:///home/felipefg/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=490950;file:///home/felipefg/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             IPython 7.14, please import from IPython display                                      
                                                                                                                   

                    WARNING  Importing display from IPython.core.display is deprecated since        ]8;id=896070;file:///home/felipefg/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=605302;file:///home/felipefg/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             IPython 7.14, please import from IPython display                                      
                                                                                                                   

                    WARNING  Importing display from IPython.core.display is deprecated since        ]8;id=563880;file:///home/felipefg/.pyenv/versions/3.10.8/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=879371;file:///home/felipefg/.pyenv/versions/3.10.8/lib/python3.10/warnings.py#109\109]8;;\
                             IPython 7.14, please import from IPython display                                      
                                                                                                                   

In [4]:
# Separando as etapas de montagem de features e classificação

In [5]:
pipeline = model.best_estimator_

In [16]:
ct = pipeline.steps[0][1]
ct

ColumnTransformer(transformers=[('tfidf', TfidfVectorizer(), 'text')])

In [17]:
clf = pipeline.steps[1][1]
clf

MultinomialNB(alpha=0.1, fit_prior=False)

In [19]:
data_test = catalog.load('data_test')

[04/05/23 20:02:21] INFO     Loading data from 'data_test' (ParquetDataSet)...                  ]8;id=362471;file:///home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=816688;file:///home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [26]:
samples = data_test[['text', 'airline_sentiment']].head(10)
samples

,text,airline_sentiment
1838,@united I assume that would be for the other 3...,negative
7032,STAHHPPP!!! RT @JetBlue: Our fleet's on fleek....,neutral
1938,@united thank you,positive
3262,@united yet again you disappoint. Sitting at I...,negative
8063,@JetBlue sis and 16 month old nephew are. Last...,negative
3367,@united 777 from SFO to HNL with ZERO entertai...,negative
831,@united UA276 sitting at EWR nearly an hour af...,negative
10822,@USAirways worst customer service. ticket was ...,negative
5731,@SouthwestAir yeah... that's not what happened...,negative
7411,"@JetBlue what's up? Three delays so far, resul...",negative


In [22]:
features = ct.transform(samples)

In [25]:
clf.predict(features)

array(['negative', 'neutral', 'positive', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative'],
      dtype='<U8')

In [36]:
explainer = shap.Explainer(clf.predict, features[1,:])

In [37]:
shap_test = explainer(features, max_evals=21615)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 shap_test = explainer(features, max_evals=21615)                                             │
│   2                                                                                              │
│                                                                                                  │
│ /home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/shap/explainers/_p │
│ ermutation.py:82 in __call__                                                                     │
│                                                                                                  │
│    79 │   │   │   │    outputs=None, silent=False):                                              │
│    80 │   │   """ Explain the output of the model on the given arguments.                        │
│    81 │   │   """                                                                                │
│ ❱  82 │   │   return super().__call__(                                                           │
│    83 │   │   │   *args, max_evals=max_evals, main_effects=main_effects, error_bounds=error_bo   │
│    84 │   │   │   outputs=outputs, silent=silent                                                 │
│    85 │   │   )                                                                                  │
│                                                                                                  │
│ /home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/shap/explainers/_e │
│ xplainer.py:266 in __call__                                                                      │
│                                                                                                  │
│   263 │   │   if callable(getattr(self.masker, "feature_names", None)):                          │
│   264 │   │   │   feature_names = [[] for _ in range(len(args))]                                 │
│   265 │   │   for row_args in show_progress(zip(*args), num_rows, self.__class__.__name__+" ex   │
│ ❱ 266 │   │   │   row_result = self.explain_row(                                                 │
│   267 │   │   │   │   *row_args, max_evals=max_evals, main_effects=main_effects, error_bounds=   │
│   268 │   │   │   │   batch_size=batch_size, outputs=outputs, silent=silent, **kwargs            │
│   269 │   │   │   )                                                                              │
│                                                                                                  │
│ /home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python3.10/site-packages/shap/explainers/_p │
│ ermutation.py:92 in explain_row                                                                  │
│                                                                                                  │
│    89 │   │   """                                                                                │
│    90 │   │                                                                                      │
│    91 │   │   # build a masked version of the model for the current input sample                 │
│ ❱  92 │   │   fm = MaskedModel(self.model, self.masker, self.link, self.linearize_link, *row_a   │
│    93 │   │                                                                                      │
│    94 │   │   # by default we run 10 permutations forward and backward                           │
│    95 │   │   if max_evals == "auto":                                                            │
│                                                                                                  │
│ /home/felipefg/.pyenv/versions/3.10.8/envs/kedro/lib/python